In [ ]:
#si ya se ha instalado estos paquetes no es necesario sino dara fallos
pip install test_tube
pip install pytorch-lightning

In [47]:
import pytorch_lightning as pl
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.datasets import fetch_openml
import numpy as np
from torch.utils.data import DataLoader


In [48]:
import os
import torch
import torchvision
import PIL.Image as Image
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [49]:
#direccion utilizada para los datasets de train y tests
train_dataset_path = 'C:/Users/eloy/Downloads/pajaros2/birds/birds'
test_dataset_path = 'C:/Users/eloy/Downloads/pajaros2/birds/birdsTest'

In [50]:
#desviacion tipica
mean = [0.4704, 0.4669, 0.3898]
#desviacion estandar
std = [0.2037, 0.2002, 0.2051]
#Estas son unas serie de transformaciones para transformarlas a 224 por 224 y ayudan a una mayor comprension de la red segun investigamos
train_transforms = transforms.Compose([
                                        transforms.Resize((224,224)),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.RandomRotation(10),
                                        transforms.ToTensor(),
                                        transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))])

test_transforms = transforms.Compose([
                                        transforms.Resize((224,224)),
                                        transforms.ToTensor(),
                                        transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))])

In [51]:
#definimos una clase para crear los loaders y todo tipo de dato necesario 
#puede cambiarse aqui el batch size 
class DataModule(pl.LightningDataModule):

    def __init__(self, batch_size: int = 64):
        super().__init__()
        self.batch_size = batch_size

    def setup(self, stage=None):
        self.train_dataset = torchvision.datasets.ImageFolder(root=train_dataset_path, transform=train_transforms)
        self.test_dataset = torchvision.datasets.ImageFolder(root= test_dataset_path, transform=test_transforms)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size)

In [52]:
#(torch.Size([tamaño del batch, numero de canales de la imagen, tamaño en xde la imagen , tamaño en y de la imagen]), torch.Size([tamaño del batch]))
#esto lo hacemos para crear los datos de las imagenes junto a sus labels
dm = DataModule()
dm.setup()
imgs, labels = next(iter(dm.train_dataloader()))
imgs.shape, labels.shape

(torch.Size([64, 3, 224, 224]), torch.Size([64]))

In [53]:
import torch.nn as nn


# creacion del PatchEmbedding que divide en parches las imagenes de entrada
class PatchEmbedding(nn.Module):
    def __init__(self, img_size, patch_size, in_chans, embed_dim):
        super().__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.n_patches = (img_size // patch_size) ** 2
        self.patch_size = patch_size
        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        x = self.proj(x)  # (B, E, P, P)
        x = x.flatten(2)  # (B, E, N)
        x = x.transpose(1, 2)  # (B, N, E)
        return x

In [54]:
#torch.Size([tamaño del batch, numero de parches, numero de valores por cada parche])
pe = PatchEmbedding(224, 32, 3, 300)
out = pe(imgs)
out.shape

torch.Size([64, 49, 300])

In [55]:
import math

#creamos el modelo de atencion, en este caso del tipo MultiHeadAttention 
class MultiHeadAttention(nn.Module):

    def __init__(self, n_embd, n_heads):
        super().__init__()
        self.n_heads = n_heads #numero de cabezas 
        
        
        # key, query, value
        
        self.key = nn.Linear(n_embd, n_embd*n_heads)
        self.query = nn.Linear(n_embd, n_embd*n_heads)
        self.value = nn.Linear(n_embd, n_embd*n_heads)
        
        # proyeccion
        self.proj = nn.Linear(n_embd*n_heads, n_embd)

    def forward(self, x):
        B, L, F = x.size()
        
        #calcula la query, key, values para todas las cabeza para todas las cabezas en batch 
        
        k = self.key(x).view(B, L, F, self.n_heads).transpose(1, 3) # (B, nh, L, F)
        q = self.query(x).view(B, L, F, self.n_heads).transpose(1, 3) # (B, nh, L, F)
        v = self.value(x).view(B, L, F, self.n_heads).transpose(1, 3) # (B, nh, L, F)
        
        # aplicamos la formula (B, nh, L, F) x (B, nh, F, L) -> (B, nh, L, L) 
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        att = torch.nn.functional.softmax(att, dim=-1)
        y = att @ v # (B, nh, L, L) x (B, nh, L, F) -> (B, nh, L, F)
        y = y.transpose(1, 2).contiguous().view(B, L, F*self.n_heads) #volvemos a ensamblar todas las salidas de la cabeza una al lado de la otra
        
        return self.proj(y)

In [56]:

#creamos el bloque encoder siguiendo el esquema de ViT
class TransformerBlock(nn.Module):
    def __init__(self, n_embd, n_heads):
        super().__init__()
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        self.attn = MultiHeadAttention(n_embd, n_heads)
        self.mlp = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
        )
        
    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.mlp(self.ln2(x))
        return x

In [57]:
#creamos la clase visual transformer siguiendo el modelo establecido para este tipo de transformer 
#aqui puedes cambiar todo los atributos para este transformer 
class ViT(nn.Module):

    def __init__(self, img_size=224, patch_size=7, in_chans=3, embed_dim=100, n_heads=3, n_layers=3, n_classes=400):
        super().__init__()

        self.patch_embed = PatchEmbedding(img_size, patch_size, in_chans, embed_dim)
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, 1 + self.patch_embed.n_patches, embed_dim))
        
        self.tranformer = torch.nn.Sequential(*[TransformerBlock(embed_dim, n_heads) for _ in range(n_layers)])
        
        self.ln = nn.LayerNorm(embed_dim)
        self.fc = torch.nn.Linear(embed_dim, n_classes)

    def forward(self, x):
        e = self.patch_embed(x)
        B, L, E = e.size()
        
        cls_token = self.cls_token.expand(B, -1, -1)  # (B, 1, E)
        e = torch.cat((cls_token, e), dim=1)  # (B, 1 + N, E)
        e = e + self.pos_embed  # (B, 1 + N, E)
        
        z = self.tranformer(e)
        
        cls_token_final = z[:, 0]  
        y = self.fc(cls_token_final)

        return y

In [58]:
#definimos el entrenamiento a seguir para el modelo
class Model(pl.LightningModule):

    def __init__(self):
        super().__init__()
        self.vit = ViT()

    def forward(self, x):
        return self.vit(x)

    def predict(self, x):
        with torch.no_grad():
          y_hat = self(x)
          return torch.argmax(y_hat, axis=1)
        
    def compute_loss_and_acc(self, batch):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        acc = (torch.argmax(y_hat, axis=1) == y).sum().item() / y.shape[0]
        return loss, acc
    
    def training_step(self, batch, batch_idx):
        loss, acc = self.compute_loss_and_acc(batch)
        self.log('loss', loss)
        self.log('acc', acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, acc = self.compute_loss_and_acc(batch)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.0003)
        return optimizer

In [59]:
#creamos el modelo con las imagenes
model = Model()
out = model(imgs)
out.shape

torch.Size([64, 400])

In [36]:
#entrenamos el modelo
model = Model()
trainer = pl.Trainer(max_epochs=10, gpus=1, logger=None)
trainer.fit(model, dm)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\eloy\anaconda3\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:611: UserWarning: Checkpoint directory C:\Users\eloy\Downloads\jupiter IA\checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type | Params
------------------------------
0 | vit  | ViT  | 763 K 
------------------------------
763 K     Trainable params
0         Non-trainable params
763 K     Total params
3.055     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

C:\Users\eloy\anaconda3\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\eloy\anaconda3\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [37]:
#aqui guardamos el modelo solo hay que darle el path
model_path = 'C:/Users/eloy/Downloads/jupiter ia/Vitmodelo.pth'
torch.save(model,model_path)


In [60]:
#aqui cargamos el modelo solo hay que darle el path
model_path = 'C:/Users/eloy/Downloads/jupiter IA/Vitmodelo.pth'
model = torch.load(model_path)

In [61]:
imgs, labels = next(iter(dm.val_dataloader()))
preds = model.predict(imgs)


In [62]:
#se crea una pequeña funcion para clasificar todo los tests
def classify(model, image_transforms, image_path, classes):
    model = model.eval()
    image = Image.open(image_path)
    image = image_transforms(image).float()
    image = image.unsqueeze(0)
    output = model(image)
    _, predicted = torch.max(output.data, 1)
    return classes[predicted.item()]

In [63]:
mean = [0.4704, 0.4669, 0.3898]
std = [0.2037, 0.2002, 0.2051]

image_transforms = transforms.Compose([
                                        transforms.Resize((224,224)),
                                        transforms.ToTensor(),
                                        transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))])

In [64]:
#cargamos clases con el numero de clases para utilizarlo mas tarde se pude cambiar para coger la carpeta birds
clases_path = 'C:/Users/eloy/Downloads/pajaros2/birds/birds'
classes = os.listdir(clases_path)

In [65]:
#se crea una lista de las imagenes de los tests que utilizamos
#cambiar el path si es necesario utilizar otros tests
tests_path = 'C:/Users/eloy/Downloads/pajaros2/submission_test/submission_test'
a = os.listdir(tests_path)
print(len(a))
a.sort(key=len)
print(a)

2000
['391.jpg', '761.jpg', '1311.jpg', '1425.jpg', '1462.jpg', '1587.jpg', '3293.jpg', '3576.jpg', '4746.jpg', '5122.jpg', '6529.jpg', '7485.jpg', '8828.jpg', '9104.jpg', '9911.jpg', '9991.jpg', '10049.jpg', '11849.jpg', '12227.jpg', '12672.jpg', '12751.jpg', '12992.jpg', '13137.jpg', '13815.jpg', '14884.jpg', '14900.jpg', '15596.jpg', '16494.jpg', '16657.jpg', '17967.jpg', '18068.jpg', '18581.jpg', '18890.jpg', '20483.jpg', '20554.jpg', '20666.jpg', '21490.jpg', '24828.jpg', '25653.jpg', '27463.jpg', '28017.jpg', '28135.jpg', '28397.jpg', '29628.jpg', '30969.jpg', '34293.jpg', '35577.jpg', '35712.jpg', '38676.jpg', '39606.jpg', '39788.jpg', '40794.jpg', '43142.jpg', '44422.jpg', '45499.jpg', '46413.jpg', '48914.jpg', '52397.jpg', '52847.jpg', '53059.jpg', '56545.jpg', '57790.jpg', '60339.jpg', '61465.jpg', '61571.jpg', '61578.jpg', '62808.jpg', '64101.jpg', '66619.jpg', '67811.jpg', '71110.jpg', '71200.jpg', '71685.jpg', '71786.jpg', '72428.jpg', '72741.jpg', '73343.jpg', '74365.jpg'

In [66]:
import pandas as pd
import numpy as np
x = 'Id , Category'
lista_Indice=[]
lista_Category=[]
tests2_path = 'C:/Users/eloy/Downloads/pajaros2/submission_test/submission_test/'
print(x)
for i in a:
    l = i.split('.')[0]
    lista_Indice.append(l)
    lista_Category.append(classify(model, image_transforms, tests2_path+i ,classes))



Id , Category


In [67]:
#creamos dos arrays con las dos listas y una tabla con ellos que pasamos a csv
array_de =np.array([lista_Indice,lista_Category])
array_de2 =np.transpose(array_de)
print(array_de)
tabla = pd.DataFrame(data=array_de2 , columns=['Id','Category'])
tabla

[['391' '761' '1311' ... '19981965' '19991139' '19991200']
 ['GREAT KISKADEE' 'AMERICAN AVOCET' 'PARAKETT  AKULET' ...
  'WHITE THROATED BEE EATER' 'IBERIAN MAGPIE' 'JANDAYA PARAKEET']]


,Id,Category
0,391,GREAT KISKADEE
1,761,AMERICAN AVOCET
2,1311,PARAKETT AKULET
3,1425,PURPLE FINCH
4,1462,BLUE COAU
...,...,...
1995,19951251,LAZULI BUNTING
1996,19961923,HOOPOES
1997,19981965,WHITE THROATED BEE EATER
1998,19991139,IBERIAN MAGPIE


In [68]:
#aqui transformamos la tabla a csv cambiar el path para guardar donde quieras
tabla_path = 'C:/Users/eloy/Downloads/jupiter ia/vitcsv.csv'
tabla.to_csv(tabla_path,index=False)
